# Bird Sound Classification
## by ABBOUDI Mohammed Amine
### 28 october 2018

#### Description of the project:
The goal is to predict the set of bird species that are present given a ten-second audio clip. This is a multi- label supervised classification problem. The training data consists of audio recordings paired with the set of species that are present.

We have 645 audio recordings, of which 322 are for training and 323 for testing.

We will start by computing the corresponding spectrograms, process them so as to extract the relevent data.
Spectrogram Images Precessing Pipeline:
    * Gaussian Filtering for smoothing and noise reduction
    * Applying a local gradient to make the bird chirps "pop out"
    * Using a binary filter to choose only the most energy dense bits of audio
    * Applying binary closing to a binary opening version of the image as a good method for noise reduction
    * Filling the hollow remaining feature
    * and removing small stains along a threshold of 100 frames.

After processing all spectrograms, we move on to syllable segmentation or feature extraction using simple framing and extraction techniques. We then dispose of the original files and only keep the segments and label them with each bird species.

Template matching is used to extract the patterns from each observation.

We then construct final training and testing dataframes, which consist of the patterns extracted, the regional clusters and the histogram off segments provided in the supplemental data.

Training the models, since 19 models are generated each fold to predict each bird species' probability to be present in each test recording. This computationally costly, so we restrict the model to the 100 most important features which are computed for each bird species, since they have different patters and therefore different significant features.

The probabilities are averaged across all folds of cross-validation to have a more robust model.

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import wave
from scipy import fft
from skimage.util import img_as_ubyte
from skimage.filters import rank
from skimage.morphology import remove_small_objects
from skimage.morphology import  disk
from skimage.feature import match_template
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

In [ ]:
essential = 'mlsp_contest_dataset/essential_data/'
supplemental = 'supplemental_data/'

WAV file reading

In [ ]:
filenames = pd.read_csv(essential+'rec_id2filename.txt')

def read_wav(filename):
    file = wave.open(filename,'r') # Init of Wave_read Object
    data = file.readframes(file.getnframes()) # returns data in hex text
    data = np.frombuffer(buffer = data, dtype = 'short') # short is the only dtype that outputs 160000 frames, the data must have been encoded with it
    file.close()
    return data


read_wav(essential+'src_wavs/'+filenames['filename'][1]+'.wav')

In [ ]:
# Reading wav files and computing spectrograms (HANNING WINDOW)
spectros = []
sr = 512 # sampling rate of each window
hann_window =  0.5*(1 -np.cos(np.array(range(sr))*2*np.pi/(sr-1) )) # Generating Hanning Windows
fr = 16000 # Framerate
length = 10 # Length of each audio clip in secs
step = 4 # window sliding step

for index in range(filenames.shape[0]):
    fft_i = []
    wav = read_wav(essential+'src_wavs/'+filenames.iloc[index,1]+'.wav')
    for j in range(int(step*length*fr/sr)-step):
        vec = wav[int(j * sr/step) : int((j+step) * sr/step)] * hann_window
        fft_i.append(abs(fft(vec,sr)[:int(sr/2)]))
    
    spectros.append(np.array(fft_i))
spectros[0].shape

In [ ]:
folds = pd.read_csv(essential+'CVfolds_2.txt')

In [ ]:
rec_label = pd.read_csv(essential+'rec_labels_test_hidden.txt', sep=';')
labels = pd.DataFrame(np.zeros((rec_label.shape[0], 19)))
for i in range(rec_label.shape[0]):
    splits = rec_label.iloc[i][0].split(',')
    del splits[0]
    for j in splits:
        if (j!='?'):
            labels.iloc[i,int(j)] = 1
labels = labels.join(folds)
labels['filename'] = filenames.filename # useful labels dataframe

In [ ]:
img = np.transpose(spectros[20])

# Deleting the lower as they correspond mainly to background noise, and bird chirps are on the higher end of the spectrum
img_crop = img[56:,:]
plt.subplot(2, 1, 1)
plt.title('Spectrogram')
plt.imshow(img_crop)
# a log version of the spectrogram appears to show much more detail, we will use both for feature extraction
img_log = np.log10(img_crop + 1e-3)
plt.subplot(2, 1, 2)
plt.title('Logarithm Spectrogram')
plt.imshow(img_log,cmap=plt.cm.afmhot_r)
plt.show

In [ ]:
# Processing Spectrograms and Extracting features 

for index in range(labels.shape[0]):
    isone = labels.iloc[index,0:19].sum() #using only recording where one and only one bird sings
    istest = labels.iloc[index,20]
    
    if (isone ==1 and istest ==0): #using only recording where one and only one bird sings from the training set
        img = np.transpose(spectros[index])
        # Deleting the lower as they correspond mainly to background noise, and bird chirps are on the higher end of the spectrum
        img_crop = img[56:,:]
        #Applying a Gaussian Filter on the spectrograms
        img_gauss = sp.ndimage.gaussian_filter(img_crop, sigma=3)
        #Local Gradient
        img_gauss_gradient = rank.gradient(img_as_ubyte((img_gauss-np.min(img_gauss) ) /(np.max(img_gauss - np.min(img_gauss))))
                                           , disk(3))
        #Applying a Binary Filter
        img_gauss_gradient_binary = img_gauss_gradient > np.percentile(img_gauss_gradient,90) # We keep only the most "energetic" values i.e. in the 90th percentile
        #Binary Closing
        img_gauss_gradient_binary_closing = sp.ndimage.binary_closing(sp.ndimage.binary_opening(img_gauss_gradient_binary))
        img_fin = sp.ndimage.binary_fill_holes(img_gauss_gradient_binary_closing)
        img_fin = remove_small_objects(img_fin,50)
        
        # a log version of the spectrogram appears to show much more detail, we will use both for feature extraction
        img_log = np.log10(img_crop + 1e-5)
        


In [ ]:
plt.figure(figsize=(10, 7))   
plt.imshow(img_fin)

### Syllable Segmentation

In [ ]:
# Processing Spectrograms and Extracting features 
segmented_feautures = []

for index in range(labels.shape[0]):
    istest = labels.iloc[index,20] #Check if observation belongs to the test set
    isone = labels.iloc[index,0:19].sum() #using only recording where one and only one bird sings

    
    if (isone ==1 and istest ==0): #using only recording where one and only one bird sings from the training set
        img = np.transpose(spectros[index])
        # Deleting the lower as they correspond mainly to background noise, and bird chirps are on the higher end of the spectrum
        img_crop = img[56:,:]
        #Applying a Gaussian Filter on the spectrograms
        img_gauss = sp.ndimage.gaussian_filter(img_crop, sigma=3)
        #Local Gradient
        img_gauss_gradient = rank.gradient(img_as_ubyte((img_gauss-np.min(img_gauss) ) /(np.max(img_gauss - np.min(img_gauss))))
                                           , disk(3))
        #Applying a Binary Filter
        img_gauss_gradient_binary = img_gauss_gradient > np.percentile(img_gauss_gradient,90) # We keep only the most "energetic" values i.e. in the 90th percentile
        #Binary Closing
        img_gauss_gradient_binary_closing = sp.ndimage.binary_closing(sp.ndimage.binary_opening(img_gauss_gradient_binary))
        img_fin = sp.ndimage.binary_fill_holes(img_gauss_gradient_binary_closing)
        img_fin = remove_small_objects(img_fin,99)
        
        
        

        label, num_features = sp.ndimage.label(img_fin)

        for id_feature in range(num_features):
            # Calculating the pixel coordinates of each feature
            feature = (label == (id_feature+1))
            feature = feature*1 # Changing True and False to 1 and 0
            ft_x = feature.max(axis =  0)
            ft_x_max = np.max(ft_x*range(len(ft_x)))
            ft_x[ft_x==0] = 2
            ft_x_min = np.argmin(ft_x)
            
            ft_y = feature.max(axis =  1)
            ft_y_max = np.max(ft_y*range(len(ft_y)))
            ft_y[ft_y==0] = 2
            ft_y_min = np.argmin(ft_y)
            
            # Cropping the spectrogram image to extract the final image of each feature (pattern)
            frame = [ft_y_min, ft_y_max, ft_x_min, ft_x_max]
            feature_img = img_gauss[ft_y_min:ft_y_max,ft_x_min:ft_x_max]
            segmented_feautures.append([index, id_feature, frame, feature_img])
            
plt.figure(figsize=(20, 15))
plt.subplot(5,1,1)
plt.title('After Local Gradient')
plt.imshow(img_gauss_gradient)
plt.subplot(5,1,2)
plt.title('After Binary Filter')
plt.imshow(img_gauss_gradient_binary)
plt.subplot(5,1,3)
plt.title('After Binary Closing')
plt.imshow(img_gauss_gradient_binary_closing)
plt.subplot(5,1,4)
plt.title('After Filling Holes')
plt.imshow(img_fin)
plt.subplot(5,1,5)
plt.title('After Labeling')
plt.imshow(label)
plt.show()

In [ ]:
# Processing Spectrograms and Extracting features 
segmented_feautures_log = []

for index in range(labels.shape[0]):
    istest = labels.iloc[index,20] #Check if observation belongs to the test set
    isone = labels.iloc[index,0:19].sum() #using only recording where one and only one bird sings

    
    if (isone ==1 and istest ==0): #using only recording where one and only one bird sings from the training set
        img = np.transpose(spectros[index])
        
        # Deleting the lower as they correspond mainly to background noise, and bird chirps are on the higher end of the spectrum
        img_crop = img[56:,:]
        
        #Applying the Log Function
        img_log = np.log10(img_crop + 1e-5)
        
        #Applying a Gaussian Filter on the spectrograms
        img_log_gauss = sp.ndimage.gaussian_filter(img_log, sigma=3)
        
        #Local Gradient
        img_log_gauss_gradient = rank.gradient(img_as_ubyte((img_log_gauss-np.min(img_log_gauss) ) /(np.max(img_log_gauss - np.min(img_log_gauss))))
                                           , disk(3)) 
                                # Makes areas of interest (which are maxima of amplitude and/or frequency more visible)
        
        #Applying a Binary Filter
        img_log_gauss_gradient_binary = (img_log_gauss_gradient > np.percentile(img_log_gauss_gradient,90)) # We keep only the most "energetic" values i.e. in the 90th percentile
        
        #Binary Closing
        img_log_gauss_gradient_binary_closing = sp.ndimage.binary_closing(sp.ndimage.binary_opening(img_log_gauss_gradient_binary))
        
        # Filling the holes present in the Spectrograms
        img_log_fin = sp.ndimage.binary_fill_holes(img_log_gauss_gradient_binary_closing)
        
        # Removing the small 'stains' still present
        img_log_fin = remove_small_objects(img_log_fin,100) 
        # We set the minimal threshold size of each segment to 100 frames
        
        
        

        label, num_features = sp.ndimage.label(img_log_fin)

        for id_feature in range(num_features):
            # Calculating the pixel coordinates of each feature
            feature = (label == (id_feature+1))
            feature = feature*1 # Changing True and False to 1 and 0
            ft_x = feature.max(axis =  0)
            ft_x_max = np.max(ft_x*range(len(ft_x)))
            ft_x[ft_x==0] = 2
            ft_x_min = np.argmin(ft_x)
            
            ft_y = feature.max(axis =  1)
            ft_y_max = np.max(ft_y*range(len(ft_y)))
            ft_y[ft_y==0] = 2
            ft_y_min = np.argmin(ft_y)
            
            # Cropping the spectrogram image to extract the final image of each feature (pattern)
            frame = [ft_y_min, ft_y_max, ft_x_min, ft_x_max]
            feature_img = img_gauss[ft_y_min:ft_y_max,ft_x_min:ft_x_max]
            segmented_feautures_log.append([index, id_feature, frame, feature_img])
            
plt.figure(figsize=(20, 15))
plt.subplot(5,1,1)
plt.title('After Local Gradient')
plt.imshow(img_log_gauss_gradient)
plt.subplot(5,1,2)
plt.title('After Binary Filter')
plt.imshow(img_log_gauss_gradient_binary)
plt.subplot(5,1,3)
plt.title('After Binary Closing')
plt.imshow(img_log_gauss_gradient_binary_closing)
plt.subplot(5,1,4)
plt.title('After Filling Holes')
plt.imshow(img_log_fin)
plt.subplot(5,1,5)
plt.title('After Labeling')
plt.imshow(label)
plt.show()

## Template Matching

In [ ]:
## SKIP IF TRAIN AND TEST FEATURES ARE PRECOMPUTED

train_features = []
train_log_features = []
test_features = []
test_log_features = []

for index in range(len(labels)):
    
    if (index%161 ==0):
        print('{}% completed..'.format(int(index/len(labels))))
    
    istest = labels.iloc[index,20]

    if(istest == 0):
        img = np.transpose(spectros[index])
        
        img_crop = img[56:,:] # Focus on the Relevant Frequency Domain
        
        img_gauss = sp.ndimage.gaussian_filter(img_crop, sigma=3) # Gaussian filter

        #Applying the Log Function
        img_log = np.log10(img_crop + 1e-5)
        
        img_log_gauss = sp.ndimage.gaussian_filter(img_log, sigma=3) # Gaussian filter
        
        all_segments = []
        for x in range(len(segmented_feautures)):
            segment = segmented_feautures [x][3]    # Getting the segmented feature
            f_min   = segmented_feautures [x][2][0] # Getting the minimal frequency of the feature
            f_max   = segmented_feautures [x][2][1] # Getting the maximal frequency of the feature
            
            # Making sure index isn't out of range
            if(f_min > 5):
                f_min = f_min-5
            else:
                f_min = 0
            
            if(f_max < 194):
                f_max = f_max+5
            else:
                f_max = 199
            
            spectrogram_part = img_gauss[f_min:f_max,:]
            # Extracting the relevant frequency interval spanning the entire X-axis of the image
            matched = match_template(spectrogram_part, segment)
            all_segments.append(np.max(matched))
            
        train_features.append(all_segments)
            
            
        all_segments = []
        for x in range(len(segmented_feautures)):
            segment = segmented_feautures [x][3]    # Getting the segmented feature
            f_min   = segmented_feautures [x][2][0] # Getting the minimal frequency of the feature
            f_max   = segmented_feautures [x][2][1] # Getting the maximal frequency of the feature
            
            # Making sure index isn't out of range
            if(f_min > 5):
                f_min = f_min-5
            else:
                f_min = 0
            
            if(f_max < 194):
                f_max = f_max+5
            else:
                f_max = 199
            
            spectrogram_part = img_gauss[f_min:f_max,:]
            # Extracting the relevant frequency interval spanning the entire X-axis of the image
            matched = match_template(spectrogram_part, segment)
            all_segments.append(np.max(matched))
        
        train_log_features.append(all_segments)
        
        
    else:
        
        img = np.transpose(spectros[index])
        
        img_crop = img[56:,:] # Focus on the Relevant Frequency Domain
        
        img_gauss = sp.ndimage.gaussian_filter(img_crop, sigma=3) # Gaussian filter

        #Applying the Log Function
        img_log = np.log10(img_crop + 1e-5)
        
        img_log_gauss = sp.ndimage.gaussian_filter(img_log, sigma=3) # Gaussian filter
        
        all_segments = []
        for x in range(len(segmented_feautures)):
            segment = segmented_feautures [x][3]    # Getting the segmented feature
            f_min   = segmented_feautures [x][2][0] # Getting the minimal frequency of the feature
            f_max   = segmented_feautures [x][2][1] # Getting the maximal frequency of the feature
            
            # Making sure index isn't out of range
            if(f_min > 5):
                f_min = f_min - 5
            else:
                f_min = 0
            
            if(f_max < 194):
                f_max = f_max + 5
            else:
                f_max = 199
            
            spectrogram_part = img_gauss[f_min:f_max,:]
            # Extracting the relevant frequency interval spanning the entire X-axis of the image
            matched = match_template(spectrogram_part, segment)
            all_segments.append(np.max(matched))
            
        test_features.append(all_segments)
            
            
        all_segments = []
        for x in range(len(segmented_feautures)):
            segment = segmented_feautures [x][3]    # Getting the segmented feature
            f_min   = segmented_feautures [x][2][0] # Getting the minimal frequency of the feature
            f_max   = segmented_feautures [x][2][1] # Getting the maximal frequency of the feature
            
            # Making sure index isn't out of range
            if(f_min > 5):
                f_min = f_min - 5
            else:
                f_min = 0
            
            if(f_max < 194):
                f_max = f_max + 5
            else:
                f_max = 199
            
            spectrogram_part = img_gauss[f_min:f_max,:] 
            # Extracting the relevant frequency interval spanning the entire X-axis of the image
            matched = match_template(spectrogram_part, segment)
            all_segments.append(np.max(matched))
        
        test_log_features.append(all_segments)


In [ ]:
## SKIP IF TRAIN AND TEST FEATURES ARE PRECOMPUTED

# Backup of Features

train_features1 = train_features
train_log_features1 = train_log_features
test_features1 = test_features
test_log_features1 = test_log_features

### Training the Model

In [ ]:
## SKIP IF TRAIN AND TEST FEATURES ARE PRECOMPUTED

# Collecting the complete training dataframe
# We will be using some of the supplemental data provided, namely the histogram of segments.

hist_segmts =  pd.read_csv(supplemental + 'histogram_of_segments.txt', sep = ',', names = ['rec_id']+['hist_'+ str(x) for x in range(100)] ,skiprows=1, header=0)

df = pd.merge(left = labels, right = hist_segmts , left_on = 'rec_id', right_on = 'rec_id', how = 'left').fillna(0)


train_features = pd.DataFrame(train_features, columns=['feature_'+str(x) for x in range(np.array(train_features).shape[1])])
train_features['rec_id']= df[df.fold==0].index

train_log_features = pd.DataFrame(train_log_features, columns=['feature_log_'+str(x) for x in range(np.array(train_log_features).shape[1])])
train_log_features['rec_id']= df[df.fold==0].index

test_features = pd.DataFrame(test_features, columns=['feature_'+str(x) for x in range(np.array(train_features).shape[1])])
test_features['rec_id']= df[df.fold==1].index

test_log_features = pd.DataFrame(test_log_features, columns=['feature_log_'+str(x) for x in range(np.array(train_log_features).shape[1])])
test_log_features['rec_id']= df[df.fold==1].index

all_train_features = pd.merge(left = train_features, right = train_log_features , left_on = 'rec_id', right_on = 'rec_id')
all_test_features  = pd.merge(left = test_features, right = test_log_features , left_on = 'rec_id', right_on = 'rec_id')

# Adding regional clustering data
region = []

for i  in range(len(df)):
    sub = df.filename.iloc[i][:7]
    sub = sub[2:sub.find('_')]
    region.append(int(sub))
    
df['Region'] = region

train = pd.merge(left = df[df.fold==0], right = all_train_features , left_index = True, right_on = 'rec_id')
test = pd.merge(left = df[df.fold==1], right = all_test_features , left_index = True, right_on = 'rec_id')

In [ ]:
# Execute this cell if csv files of the training and testing set are available.

train = pd.read_csv(supplemental+'train.csv',sep=',')
test = pd.read_csv(supplemental+'test.csv',sep=',')

train.head()

In [ ]:
cols = ['feature_'+ str(x) for x in range(322)] + ['feature_log_' + str(x) for x in range(313)] + ['hist_'+ str(x) for x in range(100)] + ['Region']


n_folds = 15
rnd = np.random.randint(0,n_folds,len(train))

# Using personalized code for cross_validation since we have a multi-label multi-instance problem.
train['cross_val'] = rnd

truth = []
prediction = []
proba = []
for i in range(19):
    proba.append(np.zeros(len(test)))


    
for n in range(n_folds):
    
    train_large = train[train.cross_val != n]
    X_train = train_large[cols]
    
    train_small = train[train.cross_val == n]
    X_test = train_small[cols]
    
    X_proba = test[cols]
    
    # Loop for generating 19 models
    for i in range(19):
        
        y_train = train_large[str(i)] # column specific to bird i in training
        y_test = train_small[str(i)] # column specific to bird i in testing
        
        best_100 = SelectKBest(f_regression,50)
        best_100.fit(X_train, y_train)
        X_train_100 = best_100.transform(X_train)
        X_test_100 = best_100.transform(X_test)
        X_proba_100 = best_100.transform(X_proba)
        
        #I tried all regressors, but the random forest performed best, followed closely by SVM regressor. 
        #I opted for RF because it allows more intuitive customizability
        
        RF = RandomForestRegressor(n_estimators = 500, max_features = 6)# using the Regressor since we are predicting a probability
        RF.fit(X_train_100,y_train)
        y_pred = RF.predict(X_test_100)
        y_proba = RF.predict(X_proba_100)

        prediction = prediction + list(y_pred)
        truth = truth + list(y_test)
        
        proba[i] = proba[i] + (y_proba/n_folds) # Averaging across all cross_validation folds

fpr, tpr, thresholds = metrics.roc_curve(truth, prediction, pos_label=1)
AUC = metrics.auc(fpr,tpr)
print('The AUC of the Model is: {}'.format(AUC) )